# First Model-1::  Model Building using food_data_db_v2.csv

### Loading data from file system

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import warnings

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder

# for postgres SQL database
import psycopg2

# Filter and ignore warnings
warnings.filterwarnings("ignore")

### Load food data

In [ ]:
# read food_data
df_food_data_sql = pd.read_csv("./application/data/food_data_db_v2.csv")

In [ ]:
df_food_data_sql.head()

,food_id,food_name,food_description,ingredients,ingredients_clean,nutrition,veg_or_non_veg,allergies,cuisine,course,diet,state,region,seasons,dietary_restrictions,data_source,created_time
0,495314,go to bbq sauce for ribs,this bbq sauce will stick to your ribs! you ca...,"{""tomato ketchup"",""dark brown sugar"",applesauc...","{""tomato ketchup"",""dark brown sugar"",applesauc...","{233.5,0.0,209.0,52.0,2.0,0.0,19.0}",non-veg,"{nightshade,oral,sugar}",NaN,NaN,NaN,NaN,NaN,{},"{""low fat""}",raw_recipes,2023-06-17 15:32:27.202
1,149593,add in anything muffins,"this is a never-fail muffin recipe, it's a bla...","{flour,sugar,""baking powder"",""baking soda"",sal...","{flour,sugar,""baking powder"",""baking soda"",sal...","{209.3,18.0,44.0,8.0,5.0,36.0,7.0}",non-veg,"{milk,lactose,sugar,dairy}",NaN,NaN,NaN,NaN,NaN,{},{},raw_recipes,2023-06-17 15:32:27.202
2,227924,almost dairy queen blizzard ice cream cake,"just wait until you taste this, it is so easy ...","{""oreo cookies"",""vanilla ice cream"",""frozen wh...","{""oreo cooky"",""vanilla ice cream"",""frozen whip...","{455.3,37.0,158.0,13.0,11.0,57.0,19.0}",veg,"{milk,lactose,dairy}",NaN,NaN,NaN,NaN,NaN,{winter},{},raw_recipes,2023-06-17 15:32:27.203
3,100540,almost grilled cheese sandwich,"good, simple sandwich.","{""white bread"",""processed cheese"",butter}","{""white bread"",""processed cheese"",butter}","{195.3,9.0,14.0,26.0,15.0,16.0,8.0}",veg,"{gluten,milk,lactose,dairy}",NaN,NaN,NaN,NaN,NaN,{},{},raw_recipes,2023-06-17 15:32:27.204
4,155959,ant kelly s london broil marinade,my niece shwana loves this! she always writes...,"{""balsamic vinegar"",""soy sauce"",""worcestershir...","{""balsamic vinegar"",""soy sauce"",""worcestershir...","{673.5,61.0,18.0,67.0,133.0,57.0,2.0}",non-veg,"{olive,soy}",NaN,NaN,NaN,NaN,NaN,{},{},raw_recipes,2023-06-17 15:32:27.205


In [ ]:
df_food_data_sql.columns

Index(['food_id', 'food_name', 'food_description', 'ingredients',
       'ingredients_clean', 'nutrition', 'veg_or_non_veg', 'allergies',
       'cuisine', 'course', 'diet', 'state', 'region', 'seasons',
       'dietary_restrictions', 'data_source', 'created_time'],
      dtype='object')

In [ ]:
import ast
# data in csv file are not in proper list format
df_food_data_sql["ingredients_clean"] = df_food_data_sql["ingredients_clean"].apply(lambda val: [elem.strip() for elem in re.sub('[{}"]', '', val).split(",")])
df_food_data_sql["nutrition"] = df_food_data_sql["nutrition"].apply(lambda val: [elem.strip() for elem in re.sub('[{}"]', '', val).split(",")])
df_food_data_sql["allergies"] = df_food_data_sql["allergies"].apply(lambda val: [elem.strip() for elem in re.sub('[{}"]', '', val).split(",")])
df_food_data_sql["seasons"] = df_food_data_sql["seasons"].apply(lambda val: [elem.strip() for elem in re.sub('[{}"]', '', val).split(",")])
df_food_data_sql["dietary_restrictions"] = df_food_data_sql["dietary_restrictions"].apply(lambda val: [elem.strip() for elem in re.sub('[{}"]', '', val).split(",")])

## Model with top 5 recommended foods basis user input

In [ ]:
#import pandas as pd
#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.metrics.pairwise import linear_kernel

#def recommend_food(user_input):
    # Filter based on veg_or_non_veg preference
 #   model_data_filtered = model_data[model_data["veg_or_non_veg"] == user_input['veg_or_non_veg']]
  #  print(f"After veg/non-veg filter: {len(model_data_filtered)} dishes left")

    # Filter out any foods that contain ingredients user is allergic to
   # for allergen in user_input['allergy']:
    #    model_data_filtered = model_data_filtered[~model_data_filtered['ingredients_str_lemmatized'].str.contains(allergen)]
     # print(f"After removing dishes containing {allergen}: {len(model_data_filtered)} dishes left")

    # Filter based on dietary restrictions
    # if user_input['dietary_restrictions'] != "no dietary restriction":
      #  model_data_filtered = model_data_filtered[model_data_filtered["dietary_restrictions"] == user_input['dietary_restrictions']]
       # print(f"After dietary restriction filter: {len(model_data_filtered)} dishes left")

    # Filter based on seasonal preference
   # model_data_filtered = model_data_filtered[model_data_filtered["seasonal_preference"] == user_input['seasonal_preference']]
   # print(f"After seasonal preference filter: {len(model_data_filtered)} dishes left")

    # Check if there's anything left
   # if len(model_data_filtered) == 0:
    #    print("No dishes left after filtering. Please try different preferences.")
     #   return None, None

    # Use TF-IDF to vectorize the ingredients_str_lemmatized
    #tfidf = TfidfVectorizer(stop_words='english')
    #model_data_filtered['ingredients_str_lemmatized'] = model_data_filtered['ingredients_str_lemmatized'].fillna('')
    #tfidf_matrix = tfidf.fit_transform(model_data_filtered['ingredients_str_lemmatized'])

    # Vectorize user's ingredients preference
    #user_pref_vector = tfidf.transform([user_input['ingredients_str_lemmatized']])

    # Calculate cosine similarity
    #cosine_similarities = linear_kernel(user_pref_vector, tfidf_matrix).flatten()

    # Get top 5 most similar items
    #top_5_indices = cosine_similarities.argsort()[:-6:-1]

    # Get corresponding cosine similarity scores
    #top_5_scores = cosine_similarities[top_5_indices]

    # Return names of the dishes and their cosine similarity scores
    #return model_data_filtered.iloc[top_5_indices]['name'], top_5_scores

# User inputs
#veg_or_non_veg = input("Please enter your preference (veg/non-veg): ")
#seasonal_preference = input("Please enter your seasonal preference: ")
#allergy = input("Please enter your allergies (separated by comma): ").split(", ")
#dietary_restrictions = input("Please enter your dietary restrictions: ")
#ingredients_str_lemmatized = input("Please enter your preferred ingredients: ")

#user_input = {
 #   'veg_or_non_veg': veg_or_non_veg,
  #  'seasonal_preference': seasonal_preference,
   # 'allergy': allergy,
   # 'dietary_restrictions': dietary_restrictions,
   # 'ingredients_str_lemmatized': ingredients_str_lemmatized
#}

#recommendations, scores = recommend_food(user_input)
#print("\nTop 5 food recommendations for you:")
#for i in range(5):
 #   print(f"{i+1}. {recommendations.iloc[i]} with a cosine similarity of {scores[i]}")


In [ ]:
columns = ["food_id", "food_name", "seasons", "ingredients_clean", "allergies", "dietary_restrictions", "veg_or_non_veg"]
model_data = df_food_data_sql[columns]

In [ ]:
model_data.head()

,food_id,food_name,seasons,ingredients_clean,allergies,dietary_restrictions,veg_or_non_veg
0,495314,go to bbq sauce for ribs,[],"[tomato ketchup, dark brown sugar, applesauce,...","[nightshade, oral, sugar]",[low fat],non-veg
1,149593,add in anything muffins,[],"[flour, sugar, baking powder, baking soda, sal...","[milk, lactose, sugar, dairy]",[],non-veg
2,227924,almost dairy queen blizzard ice cream cake,[winter],"[oreo cooky, vanilla ice cream, frozen whipped...","[milk, lactose, dairy]",[],veg
3,100540,almost grilled cheese sandwich,[],"[white bread, processed cheese, butter]","[gluten, milk, lactose, dairy]",[],veg
4,155959,ant kelly s london broil marinade,[],"[balsamic vinegar, soy sauce, worcestershire s...","[olive, soy]",[],non-veg


In [ ]:
# model expect comma seperate element in string form
model_data["ingredients_clean"] = model_data["ingredients_clean"].apply(lambda x_list: ", ".join(x_list))
model_data["dietary_restrictions"] = model_data["dietary_restrictions"].apply(lambda x_list: ", ".join(x_list))
model_data["seasons"] = model_data["ingredients_clean"].apply(lambda x_list: ", ".join(x_list))
model_data.head()

,food_id,food_name,seasons,ingredients_clean,allergies,dietary_restrictions,veg_or_non_veg
0,495314,go to bbq sauce for ribs,"t, o, m, a, t, o, , k, e, t, c, h, u, p, ,, ...","tomato ketchup, dark brown sugar, applesauce, ...","[nightshade, oral, sugar]",low fat,non-veg
1,149593,add in anything muffins,"f, l, o, u, r, ,, , s, u, g, a, r, ,, , b, a...","flour, sugar, baking powder, baking soda, salt...","[milk, lactose, sugar, dairy]",,non-veg
2,227924,almost dairy queen blizzard ice cream cake,"o, r, e, o, , c, o, o, k, y, ,, , v, a, n, i...","oreo cooky, vanilla ice cream, frozen whipped ...","[milk, lactose, dairy]",,veg
3,100540,almost grilled cheese sandwich,"w, h, i, t, e, , b, r, e, a, d, ,, , p, r, o...","white bread, processed cheese, butter","[gluten, milk, lactose, dairy]",,veg
4,155959,ant kelly s london broil marinade,"b, a, l, s, a, m, i, c, , v, i, n, e, g, a, r...","balsamic vinegar, soy sauce, worcestershire sa...","[olive, soy]",,non-veg


## Making the model flexible with scores in case of varied user preferences

In [ ]:
def recommend_food(user_input):
    # Create a score for each dish
    scores = np.zeros(len(model_data))

    # Give higher score to dishes that match veg_or_non_veg preference
    scores[model_data["veg_or_non_veg"] == user_input['veg_or_non_veg']] += 1

    # Give lower score to dishes that contain ingredients user is allergic to
    for allergen in user_input['allergy']:
        scores[model_data['ingredients_clean'].str.contains(allergen)] -= 0.5

    # Give higher score to dishes that match dietary restrictions
    if user_input['dietary_restrictions'] != "no dietary restriction":
        scores[model_data["dietary_restrictions"] == user_input['dietary_restrictions']] += 1

    # Give higher score to dishes that match seasonal preference
    scores[model_data["seasons"].isin(user_input['seasonal_preference'])] += 1

    # Use TF-IDF to vectorize the ingredients_str_lemmatized
    tfidf = TfidfVectorizer(stop_words='english')
    model_data['ingredients_clean'] = model_data['ingredients_clean'].fillna('')
    tfidf_matrix = tfidf.fit_transform(model_data['ingredients_clean'])

    # Vectorize user's ingredients preference
    user_pref_vector = tfidf.transform([user_input['ingredients_str_lemmatized']])

    # Calculate cosine similarity and add to scores
    cosine_similarities = linear_kernel(user_pref_vector, tfidf_matrix).flatten()
    scores += cosine_similarities

    # Get top 10 highest scoring dishes
    top_10_indices = scores.argsort()[:-11:-1]

    # Get corresponding cosine similarity scores
    top_10_scores = cosine_similarities[top_10_indices]

    # Return names of the dishes and their scores
    return model_data.iloc[top_10_indices], top_10_scores

In [ ]:
# User inputs
veg_or_non_veg = input("Please enter your preference (veg/non-veg): ")
seasonal_preference = input("Please enter your seasonal preference (separated by comma): ").split(", ")
allergy = input("Please enter your allergies (separated by comma): ").split(", ")
dietary_restrictions = input("Please enter your dietary restrictions: ")
ingredients_str_lemmatized = input("Please enter your preferred ingredients: ")

user_input = {
    'veg_or_non_veg': veg_or_non_veg,
    'seasonal_preference': seasonal_preference,
    'allergy': allergy,
    'dietary_restrictions': dietary_restrictions,
    'ingredients_str_lemmatized': ingredients_str_lemmatized
}

recommendations, scores = recommend_food(user_input)

print(f"\nList of food-ids: {list(recommendations['food_id'])}")

print("\nTop ten food recommendations for you:")
for i in range(10):
    print(f"{i+1}. {recommendations.iloc[i]['food_id']}, {recommendations.iloc[i]['food_name']} with a cosine similarity of {scores[i]}")

Please enter your preference (veg/non-veg): veg
Please enter your seasonal preference (separated by comma): winter
Please enter your allergies (separated by comma): nut
Please enter your dietary restrictions: milk
Please enter your preferred ingredients: rice

List of food-ids: [250026, 335754, 164741, 600444, 339010, 39269, 234759, 700047, 16465, 339009]

Top ten food recommendations for you:
1. 250026, no fail foolproof   rice with a cosine similarity of 0.8159473867022633
2. 335754, perfect rice 2 cups with a cosine similarity of 0.8159473867022633
3. 164741, pressure cooker rice with a cosine similarity of 0.7684255749626663
4. 600444, Rice   Steamed Rice  with a cosine similarity of 0.7684255749626663
5. 339010, himalayan red rice with a cosine similarity of 0.7684255749626663
6. 39269, mirj s foolproof microwave rice   perfect every time with a cosine similarity of 0.7684255749626663
7. 234759, simple microwave cooked rice with a cosine similarity of 0.7332038062911125
8. 700047,

In [ ]:
user_input

{'veg_or_non_veg': 'veg',
 'seasonal_preference': ['winter'],
 'allergy': ['nut'],
 'dietary_restrictions': 'milk',
 'ingredients_str_lemmatized': 'rice'}